# 1. Create bash script to copy instrument PDFs

In [34]:
import os
import glob
from bs4 import BeautifulSoup
import re
import PyPDF2
import shutil
import random
from collections import defaultdict
from pathlib import Path
import string

In [35]:
def isValidPDF(infile):
    isValid = True
    try:
        PyPDF2.PdfFileReader(open(infile, "rb"))
    except:
        isValid = False
    return isValid

In [36]:
def parsePdfInfo(string):
    # an example pdf info string:
    # '*#400812 - 22.72MB, 8 pp. - 0.0/10\n\n2\n4\n6\n8\n10\n\n (-) - V/V/V - 2151×⇩ - Piupianissimo'
    string = string.replace('\n','')
    m = re.search('#(\d+)\s', string)
    pdf_id = m.group(1)
    m = re.search('-\s(\d+)×', string)
    if m:
        num_downloads = int(m.group(1))
    else:
        num_downloads = 0 # no downloads field
    return pdf_id, num_downloads

In [37]:
def parseHtml(html_file):
    with open(html_file,'r') as f:
        text = f.read()
    soup = BeautifulSoup(text, 'html.parser')
    return soup

In [38]:
def extractScoreInfo(node):
    
    # read all table entries
    d = defaultdict(str)
    for row in node.find("table").find("table").findAll("tr"):
        row_header = row.find("th").text.strip().lower()
        row_field = row.find("td").text.strip().lower()
        d[row_header] = row_field
#         print('{}|{}'.format(row_header, row_field))
    
    # extract relevant fields
    publisherInfo = d['publisher. info.']
    copyright = d['copyright']
    
    # check if valid
    isPublicDomain = ('public domain' in copyright) and ('non-pd' not in copyright)
    isCreativeCommons = 'creative commons' in copyright
    isShareable = isPublicDomain or isCreativeCommons
    isManuscript = 'manuscript' in publisherInfo
    
    return isShareable, isManuscript

In [47]:
def is_solo_piece(soup, instruments):
    try:
        category_tags = soup.find(class_ = "wp_header").find("table").findAll("span", class_ = "plainlinks")
    except:
        return (False, "")
    for tag in category_tags:
        category = tag.text  # we want categories of the form "For (instrument)"
        if category.startswith("For"):
            words = category.lower().split()
            if len(words) == 2 and words[1] in instruments:
                return (True, words[1])
             # augment data with trumpet + piano, trumpet + orchestra data
            if category in ["For trumpet, piano", "For trumpet, orchestra"]: 
                return (True, "trumpet")
            if category == "For oboe, orchestra":
                return (True, "oboe")
    
    return (False, "")

In [48]:
def selectSinglePDF(html_file):
    '''
    Selects a single PDF file from an html metadata file.  First check if the piece category is one of the desired, categories, then
    filter the list of pdfs to ensure that the piece has a suitable copyright and is not a manuscript,and finally,
    within the remainining options select the most popular (i.e. most downloaded) score.  
    
    Returns the full path to the selected pdf file and the instrument. If no valid pdfs are found, returns None.
    '''    # get pdf ids, number of downloads
    soup = parseHtml(html_file)
    is_solo, instrument = is_solo_piece(soup, instruments)
    if not is_solo:
        return "", ""
    scores_list = soup.find_all(class_ = 'we')
    if scores_list is None: # no scores
        return "", ""
    tuples = [] # populate with (pdf_id, num_downloads)
    for i, child in enumerate(scores_list): # list of scores
        try:
            isShareable, isManuscript = extractScoreInfo(child) # copyright ok & not a manuscript
        except: # incorrectly formatted entry -- skip
            continue
        for pdf_div_tag in child.find_all("div", recursive=False): # may have multiple pdfs
            try:
                title = pdf_div_tag.find("span", {"title": "Download this file"}).text.lower()
            except:
                continue
            we_file_info2 = pdf_div_tag.find(class_='we_file_info2')
            if we_file_info2 is not None: # sometimes there are div tags with additional information
                pdf_id, num_downloads = parsePdfInfo(we_file_info2.text)
                tuples.append((pdf_id, num_downloads, isShareable, not isManuscript))
    # sort by downloads, verify that PDF can be read
    valid_list = []
    tuples.sort(key = lambda x: x[1], reverse = True)
    piece_dir = html_file[:-9]
    for (pdf_id, num_downloads, isPublicDomain, notManuscript) in tuples:
        fullpath = '{}/{}.pdf'.format(piece_dir, pdf_id)
        # make sure pieces are publicly available and are not manuscripts
        # return the PDF with the most downloads
        if isPublicDomain and notManuscript and isValidPDF(fullpath):
            return fullpath, instrument.split()[0]    # make sure oboe/trumpet + others map to the correct instrument
    return "", ""

In [49]:
instruments = ["cello", "clarinet", "flute", "oboe", "trumpet", "viola", "violin", "guitar"]

In [50]:
outfile = '/home/kji/InstrumentID/copy_executed.sh'
imslp_dir = Path('/data/Datasets/imslp/score_scrape/results/composer/')
data_dir = '/home/kji/InstrumentID/data/all/'

In [24]:
# imslp_dir = Path("/data/Datasets/imslp/score_scrape/results/composer/Rondeau,_Michel/Trumpet_Sonata_No.3_in_E_minor_")

In [ ]:
# if not os.path.exists(outfile):
    with open(outfile, 'w') as f:
        instrument_counts = defaultdict(int)
        for file in imslp_dir.rglob("html.txt"):
            file = str(file)
            orig_path, instrument = selectSinglePDF(file)
            if orig_path != "":
                instrument_counts[instrument] += 1
                new_path = f"{data_dir}{instrument}/{instrument}_{instrument_counts[instrument]}.pdf"
                f.write(f"cp {orig_path} {new_path}\n")

# 2. Execute the bash script to populate directories

Execute the script ```copy_executed.sh``` using ```bash copy_executed.sh```.

In [ ]:
cp /data/Datasets/imslp/score_scrape/results/composer/La_Laurencie,_Lionel_de/L%27%C3%A9cole_fran%C3%A7aise_de_violon,_de_Lully_%C3%A0_Viotti_%C3%A9tudes_d%27histoire_et_d%27esth%C3%A9tique_/71976.pdf /home/kji/InstrumentID/data/all/violin/violin_90

In [ ]:
/data/Datasets/imslp/score_scrape/results/composer/Zimmermann_(guitar_composer),_H./L%C3%A4ndler_/44206.pdf /home/kji/InstrumentID/data/all/guitar/guitar_529

# 3. Randomly select 75 PDFs per instrument to be manually labeled

In [52]:
dest_dir = '/home/kji/InstrumentID/data/labeled/'
samples = 75
random.seed(2)

In [68]:
def random_instrument_sample(src_dir, dest_dir, num_samples):
    filenames = random.sample(os.listdir(src_dir), num_samples)
    filenames = os.listdir(dest_dir)
    for name in filenames:
        src_path = os.path.join(src_dir, name)
        shutil.copy(src_path, dest_dir)

In [69]:
for instrument in instruments:
    src_dir = data_dir + instrument
    dest = dest_dir + instrument
    random_instrument_sample(src_dir, dest, samples)

['cello_221.pdf', 'cello_218.pdf', 'cello_15.pdf', 'cello_24.pdf', 'cello_22.pdf', 'cello_93.pdf', 'cello_214.pdf', 'cello_44.pdf', 'cello_189.pdf', 'cello_208.pdf', 'cello_172.pdf', 'cello_79.pdf', 'cello_65.pdf', 'cello_156.pdf', 'cello_55.pdf', 'cello_213.pdf', 'cello_10.pdf', 'cello_149.pdf', 'cello_175.pdf', 'cello_41.pdf', 'cello_111.pdf', 'cello_164.pdf', 'cello_101.pdf', 'cello_186.pdf', 'cello_131.pdf', 'cello_96.pdf', 'cello_140.pdf', 'cello_114.pdf', 'cello_129.pdf', 'cello_69.pdf', 'cello_210.pdf', 'cello_8.pdf', 'cello_94.pdf', 'cello_120.pdf', 'cello_82.pdf', 'cello_98.pdf', 'cello_109.pdf', 'cello_135.pdf', 'cello_43.pdf', 'cello_144.pdf', 'cello_46.pdf', 'cello_61.pdf', 'cello_60.pdf', 'cello_7.pdf', 'cello_203.pdf', 'cello_84.pdf', 'cello_45.pdf', 'cello_35.pdf', 'cello_202.pdf', 'cello_178.pdf', 'cello_226.pdf', 'cello_132.pdf', 'cello_173.pdf', 'cello_187.pdf', 'cello_47.pdf', 'cello_115.pdf', 'cello_107.pdf', 'cello_225.pdf', 'cello_194.pdf', 'cello_152.pdf', 'cello

['guitar_326.pdf', 'guitar_204.pdf', 'guitar_488.pdf', 'guitar_961.pdf', 'guitar_389.pdf', 'guitar_1390.pdf', 'guitar_766.pdf', 'guitar_380.pdf', 'guitar_730.pdf', 'guitar_288.pdf', 'guitar_278.pdf', 'guitar_478.pdf', 'guitar_551.pdf', 'guitar_1128.pdf', 'guitar_1297.pdf', 'guitar_775.pdf', 'guitar_820.pdf', 'guitar_701.pdf', 'guitar_575.pdf', 'guitar_1475.pdf', 'guitar_1219.pdf', 'guitar_1030.pdf', 'guitar_1190.pdf', 'guitar_1414.pdf', 'guitar_1500.pdf', 'guitar_657.pdf', 'guitar_819.pdf', 'guitar_1462.pdf', 'guitar_1443.pdf', 'guitar_1295.pdf', 'guitar_1468.pdf', 'guitar_598.pdf', 'guitar_1090.pdf', 'guitar_1275.pdf', 'guitar_1305.pdf', 'guitar_1374.pdf', 'guitar_149.pdf', 'guitar_753.pdf', 'guitar_632.pdf', 'guitar_810.pdf', 'guitar_992.pdf', 'guitar_358.pdf', 'guitar_529.pdf', 'guitar_725.pdf', 'guitar_903.pdf', 'guitar_977.pdf', 'guitar_180.pdf', 'guitar_382.pdf', 'guitar_645.pdf', 'guitar_777.pdf', 'guitar_261.pdf', 'guitar_58.pdf', 'guitar_214.pdf', 'guitar_719.pdf', 'guitar_343